### Algoritmos de optimización - Trabajo Práctico
    Nombre y Apellidos: Sergio Moya Rodrigo
    https://github.com/smoyarodrigo/Trabajo-Practico-Sergio_Moya
    
## Problema

    SESIONES DE DOBLAJE 

    Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio d grabación independientemente del número de tomas que se graben. No es posible grabar s de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por os servicios de los actores de doblaje sea el menor posible. Los datos son:
    
    Número de actores : 10
    Número de tomas : 30
    Actores/Tomas : https://bit.ly/36D8IuK (Datos problema doblaje 30 tomas, 10 actores)

### MODELO 

- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

  

    Es un porblema coplejo que puede adoptar diferentes técnicas de tratarlo.

    Para representar el espacio de soluciones podemos utilizar un vector de longitud 30 (una posición por toma) donde cada elemento indica el día asignado a esa toma. Por ejemplo, [1, 1, 2, 3, ...].

    tenemos las restricciones de 6 tomas/día: como máximo . Los días se numeran secuencialmente (1, 2, 3, ...), y el vector debe cumplir que ningún día tenga más de 6 tomas asignadas.

  nuestra función objetivo debe minimizar el costo total, que es la suma de los días que cada actor debe asistir al estudio. Para ello debemos calcular que para cada actor, se cuenta en cuántos días diferentes aparece en las tomas asignadas sumando estos valores para los 10 actores.

  Restricciones, máximos 6 tomas por día sería la primera, y la segunda tomas asignadas a un sólo día 

### ANÁLISIS 

    ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

    El problema puede ser entendido o asemejado al de empaquetamiento combinado con restricciones de superposición de actores, lo que lo puede llegar a convertir en algo similar a NP-duro, ya que se trta de minimizar el número de días para asignar 30 tomas con una capacidad máximade 6/día. Además añadiendo una restricción más, minimizar los días de asistencia

    El espacio de soluciones válidas corresponde al número de formas de distribuir 30 tomas en días, respetando el límite de 6 tomas/día. [30/6] = 5, número mínimo de días, el máximo serían 30. Espacio total ∑ desde k = 5 hasta 30 combinaciones para dividir 30 tomas en k días. con algoritmos genéticos podíamos evitar evaluar todas las soluciones.


    

### Diseño

    Qué técnica utilizo y por que

    Voy a utilizar (AG), Algortimos genéticos porque como hemos coemntado anteriormente me van a evitar evaluar todas las soluciones acercándome a la solcuión óptima con mecanismos de exploración y mejora iterativa. Trabajando con múltiples soluciones al ismo tiempo evitando quedarme en óptimos locales. Creo que nos puede propocionar una solución robusta y eficiente por esa capacidad de manejar restricciones y búsquedas complejas adaptando la función objetivo para priorizar agrupaciones que comparten actores y reduciendo los días de desplazamiento 

### DESARROLLO CÓDIGO 

In [1]:
import deap ## hemos tenido que importar DEAP para implementar algoritmos evolutivos como los genétios (AG)
print("¡DEAP funciona correctamente!")

¡DEAP funciona correctamente!


In [2]:
conda install -c conda-forge deap

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE
failed

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package deap-1.3.3-py311h8b32b4d_1 requires libstdcxx-ng >=12, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ deap is installable with the potential options
│  ├─ deap [1.3.3|1.4.1] would require
│  │  └─ libstdcxx-ng >=12 , which can be installed;
│  ├─ deap [1.0.2.post2|1.2.2|1.3.0|1.3.1] would require
│  │  └─ python [2.7* |>=2.7,<2.8.0a0 ], which can be installed;
│  ├─ deap 1.0.2.post2 would require
│  │  └─ python 3.4* , which can be installed;
│  ├─ deap [1.0.2.post2|1.2.2] would require
│  │  └─ python [3.5* |>=3.5,<3.6.0a0 ], which can be installed;
│  ├─ deap 1.2.2 would require
│  │  └─ python 3.6* , which can be installed;
│  ├─ deap [1.2.2|1.3.0|1.3.1] would require
│ 

In [3]:
pip install deap

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [6]:
import random
from deap import base, creator, tools, algorithms
import numpy as np

# Para las tomas (Actor 1 al 10  como 0-9)
tomas = [
    [0,1,2,3,4], [2,3,4], [1,4,6], [0,1,6,7], [1,3,7],
    [0,1,3,4], [0,1,3,4], [0,1,5], [0,1,3], [0,1,5,8],
    [0,1,2,4,7], [0,1,2,3,5], [0,3,4], [0,2,5], [0,1,6],
    [3,9], [0,2], [2,5], [0,2], [0,2,3,4], [5,7],
    [0,1,2,3], [0,2], [2,5], [0,1,3,9], [0,2,4,8],
    [3,4], [0,3], [0,4,5], [0,3]
]

# Configuración de DEAP qeu maneja el prpoblema de forma flexible y eficiente 
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_day", random.randint, 0, 29)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_day, n=30)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    actor_days = [set() for _ in range(10)]
    for toma_idx, day in enumerate(individual):
        for actor in tomas[toma_idx]:
            actor_days[actor].add(day)
    total_days = sum(len(days) for days in actor_days)
    
    day_counts = {}
    for day in individual:
        day_counts[day] = day_counts.get(day, 0) + 1
    penalty = sum((count - 6) * 1000 for count in day_counts.values() if count > 6)
    return (total_days + penalty,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=29, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(42)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, 
                                   stats=stats, halloffame=hof, verbose=True)
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()
    best_ind = hof[0]
    print("Mejor asignación de días:", best_ind)
    print("Fitness (total días + penalización):", best_ind.fitness.values[0])
    
    # Detalles de la solución
    day_counts = {}
    for day in best_ind:
        day_counts[day] = day_counts.get(day, 0) + 1
    print("\nTomas por día:", day_counts)
    
    actor_days = [set() for _ in range(10)]
    for toma_idx, day in enumerate(best_ind):
        for actor in tomas[toma_idx]:
            actor_days[actor].add(day)
    print("\nDías por actor:", [len(d) for d in actor_days])

gen	nevals	avg  	min	max
0  	100   	76.56	64 	86 
1  	68    	74.3 	64 	84 
2  	64    	72.05	64 	83 
3  	57    	79.97	62 	1062
4  	71    	78.21	56 	1061
5  	65    	66.74	56 	78  
6  	55    	74.74	55 	1064
7  	54    	62.85	54 	77  
8  	66    	79.88	51 	1056
9  	69    	57.07	52 	70  
10 	49    	55.78	52 	64  
11 	65    	94.73	52 	1057
12 	58    	54.04	51 	65  
13 	53    	83.4 	51 	2051
14 	56    	72.69	50 	1055
15 	68    	72.36	50 	1049
16 	61    	72.35	50 	1054
17 	57    	71.61	49 	1053
18 	74    	72.06	47 	1056
19 	60    	81.07	47 	1057
20 	58    	90.73	47 	1061
21 	46    	49.39	47 	55  
22 	67    	109.34	46 	1056
23 	58    	88.44 	45 	1056
24 	56    	107.81	46 	2047
25 	69    	98.12 	46 	1049
26 	61    	77.8  	45 	1050
27 	67    	67.23 	45 	1054
28 	66    	67.06 	44 	1046
29 	64    	96.64 	44 	1050
30 	54    	76.28 	43 	1054
31 	67    	46.11 	43 	57  
32 	69    	75.1  	43 	1050
33 	64    	45.27 	43 	59  
34 	60    	74.35 	43 	1052
35 	60    	53.64 	43 	1046
36 	63    	43.62 	42 	55  
3

### ¿Es óptima?

    Sí, pero quizá podría mejorarse, si el actor 1 por ejemplo aparece en tomas asignadas a los días 29 , 14 , 5 ,  etc. ___


In [5]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Lista CORRECTA de actores por toma (30 elementos, índices 0-29)
tomas = [
    [0,1,2,3,4],       # Toma 1 del Excel (índice 0)
    [2,3,4],            # Toma 2 (índice 1)
    [1,4,6],            # Toma 3 (índice 2)
    [0,1,6,7],          # Toma 4 (índice 3)
    [1,3,7],            # Toma 5 (índice 4)
    [0,1,3,4],          # Toma 6 (índice 5)
    [0,1,3,4],          # Toma 7 (índice 6)
    [0,1,5],            # Toma 8 (índice 7)
    [0,1,3],            # Toma 9 (índice 8)
    [0,1,5,8],          # Toma 10 (índice 9)
    [0,1,2,4,7],        # Toma 11 (índice 10)
    [0,1,2,3,5],        # Toma 12 (índice 11)
    [0,3,4],            # Toma 13 (índice 12)
    [0,2,5],            # Toma 14 (índice 13)
    [0,1,6],            # Toma 15 (índice 14)
    [3,9],              # Toma 16 (índice 15)
    [0,2],              # Toma 17 (índice 16)
    [2,5],              # Toma 18 (índice 17)
    [0,2],              # Toma 19 (índice 18)
    [0,2,3,4],          # Toma 20 (índice 19)
    [5,7],              # Toma 21 (índice 20)
    [0,1,2,3],          # Toma 22 (índice 21)
    [0,2],              # Toma 23 (índice 22)
    [2,5],              # Toma 24 (índice 23)
    [0,1,3,9],          # Toma 25 (índice 24)
    [0,2,4,8],          # Toma 26 (índice 25)
    [3,4],              # 27 (índice 26)
    [0,3],              # Toma 28 (índice 27)
    [0,4,5],            # Toma 29 (índice 28)
    [0,3],              # Toma 30 (índice 29)
]

# ----------------------------------------------
# Configuración de DEAP (no cambia)
# ----------------------------------------------
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_day", random.randint, 0, 29)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_day, n=30)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    actor_days = [set() for _ in range(10)]  # 10 actores
    for toma_idx, day in enumerate(individual):
        for actor in tomas[toma_idx]:  # Ahora tomas tiene 30 elementos
            actor_days[actor].add(day)
    total_days = sum(len(days) for days in actor_days)
    
    day_counts = {}
    for day in individual:
        day_counts[day] = day_counts.get(day, 0) + 1
    penalty = sum((count - 6) * 1000 for count in day_counts.values() if count > 6)
    return (total_days + penalty,)

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=29, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(42)
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    pop, log = algorithms.eaSimple(
        pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=200, 
        stats=stats, halloffame=hof, verbose=True
    )
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()

gen	nevals	avg  	min	max
0  	100   	76.56	64 	86 
1  	77    	75.09	66 	83 
2  	72    	73.04	65 	84 
3  	87    	72.13	64 	83 
4  	80    	69.63	64 	83 
5  	69    	98.41	62 	2071
6  	75    	76.92	58 	1065
7  	86    	85.91	58 	1063
8  	69    	64.46	56 	75  
9  	78    	92.76	57 	1066
10 	77    	91.46	56 	1063
11 	74    	110.3	55 	1062
12 	86    	98.93	53 	1059
13 	80    	147.98	53 	1062
14 	86    	77.75 	53 	1054
15 	73    	87.05 	52 	1058
16 	81    	96.43 	50 	1060
17 	73    	55.87 	51 	65  
18 	80    	65.34 	49 	1059
19 	78    	104.34	49 	1060
20 	78    	103.85	49 	1062
21 	86    	163.17	48 	2049
22 	80    	82.3  	46 	1054
23 	71    	110.86	46 	1050
24 	72    	81.09 	46 	1050
25 	82    	100.49	46 	1049
26 	85    	100.36	45 	1054
27 	81    	79.4  	44 	1052
28 	83    	108.3 	44 	1052
29 	78    	48.01 	44 	58  
30 	70    	57.39 	43 	1045
31 	82    	76.91 	43 	1049


/home/e2c672ff-af9c-4a4a-a423-65c9ca1cf82b/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/e2c672ff-af9c-4a4a-a423-65c9ca1cf82b/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


32 	84    	86.43 	43 	1058
33 	80    	85.99 	43 	1052
34 	80    	46.33 	43 	61  
35 	77    	95.16 	43 	2047
36 	79    	75.56 	43 	1051
37 	76    	65.16 	43 	1054
38 	81    	74.92 	43 	2048
39 	77    	64.79 	43 	1049
40 	85    	55.07 	43 	1042
41 	86    	55.51 	43 	1048
42 	84    	44.2  	43 	56  
43 	74    	45.08 	43 	58  
44 	87    	55.54 	43 	1046
45 	71    	64.74 	43 	1046
46 	78    	44.9  	41 	61  
47 	79    	104.93	41 	1052
48 	83    	105.06	41 	2047
49 	85    	105.23	41 	1048
50 	83    	54.98 	41 	1050
51 	87    	54.95 	41 	1049
52 	74    	74.03 	41 	1046
53 	83    	74.04 	41 	1047
54 	82    	72.87 	41 	1048
55 	78    	53.37 	41 	1045
56 	85    	43.79 	40 	56  
57 	80    	43.63 	40 	57  
58 	89    	42.8  	39 	58  
59 	81    	62.84 	39 	1045
60 	78    	62.69 	39 	1054
61 	76    	72.14 	39 	1049
62 	84    	92.24 	39 	1046
63 	80    	62.33 	39 	1052
64 	80    	81.37 	39 	1044
65 	89    	71.81 	39 	1051
66 	87    	132.23	39 	1054
67 	78    	91.62 	39 	1048
68 	85    	81.75 	39 	1050
6